In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn import preprocessing 
import seaborn as sns

In [2]:
# Read in data
filepath = '/Users/maureenkeenan/Desktop/Kaggle/Titanic'
file_train = 'train.csv'
file_test = 'test.csv'
sample = pd.read_csv(os.path.join(filepath,file_train))
data = pd.read_csv(os.path.join(filepath,file_test))
sample_original = sample
data_original = data

In [5]:
missing_val_count_by_column = (sample.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Age         177
Cabin       687
Embarked      2
dtype: int64


In [4]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
sample = pd.DataFrame(my_imputer.fit_transform(sample))
data = pd.DataFrame(my_imputer.transform(data))

# Imputation removed column names; put them back
sample.columns = sample_original.columns
data.columns = data_original.columns

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [ ]:
# Explore data
sample.describe()
sample.columns

In [4]:
y = sample.Survived # What we are fitting for
sample.groupby('Survived').Survived.count()

Survived
0    549
1    342
Name: Survived, dtype: int64

In [5]:
import nbimporter
import DataCleanup as dc

Importing Jupyter notebook from DataCleanup.ipynb


In [6]:
sample = dc.transform_features(sample)

DataCleanup.ipynb:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "    for n in range(len(Surname)):\n",


In [7]:
dc.encode_features(sample)

(     PassengerId  Survived  Pclass  \
 0              1         0       3   
 1              2         1       1   
 2              3         1       3   
 3              4         1       1   
 4              5         0       3   
 5              6         0       3   
 6              7         0       1   
 7              8         0       3   
 8              9         1       3   
 9             10         1       2   
 10            11         1       3   
 11            12         1       1   
 12            13         0       3   
 13            14         0       3   
 14            15         0       3   
 15            16         1       2   
 16            17         0       3   
 17            18         1       2   
 18            19         0       3   
 19            20         1       3   
 20            21         0       2   
 21            22         1       2   
 22            23         1       3   
 23            24         1       1   
 24            25        

In [8]:
# Do a quick subset of features
quick_features = ['Sex','Age','Pclass','SibSp','Cabin','TitleKey','Fare','Embarked']
X = sample.drop(columns = ['Survived'])
y = sample.Survived

In [9]:
# Define subfeatures
features_1 = ['Sex','Age','Pclass']
features_2 = ['Sex','Age','Pclass','SibSp','Parch']
features_3 = ['Sex','Age','Pclass','Family']
features_4 = ['Sex','Age','Pclass','SibSp','Parch','Fare']
features_5 = ['Sex','Age','Pclass','SibSp','Parch','Fare','Title']

features = [features_1, features_2, features_3, features_4, features_5]

In [10]:
# Break off validation set from training data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=0)

print(len(X_train))

712


In [11]:
# import the class
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# Create base model
model = LogisticRegression(solver='lbfgs',C = 1,max_iter=300)

for i in range(0, len(features)):
    model.fit(X_train[features[i]], y_train)
    y_pred = model.predict(X_test[features[i]])
    
    # get stats
    f1 = metrics.f1_score(y_test,y_pred, pos_label=1)
    cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
    print("Model ",i+1,"F1: ", f1.round(2), " CNF: \n", cnf_matrix)
    

Model  1 F1:  0.79  CNF: 
 [[97 13]
 [16 53]]
Model  2 F1:  0.74  CNF: 
 [[96 14]
 [20 49]]
Model  3 F1:  0.77  CNF: 
 [[97 13]
 [18 51]]
Model  4 F1:  0.75  CNF: 
 [[95 15]
 [19 50]]
Model  5 F1:  0.75  CNF: 
 [[96 14]
 [19 50]]


In [12]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred).round(2))
print("Precision:",metrics.precision_score(y_test, y_pred).round(2))
print("Recall:",metrics.recall_score(y_test, y_pred).round(2))

Accuracy: 0.82
Precision: 0.78
Recall: 0.72


In [13]:
differences = y_pred-y_test
FN = differences[differences == -1].index
sample.iloc[FN,:]


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,TitleKey,Surname,Family
298,299,1,1,"Saalfeld, Mr. Adolphe",2,4,0,0,19988,3,2,S,11,4,530,1
484,485,1,1,"Bishop, Mr. Dickinson H",2,5,1,0,11967,1,1,C,11,4,60,2
587,588,1,1,"Frolicher-Stehli, Mr. Maxmillian",2,0,1,1,13567,1,1,C,11,4,194,1
740,741,1,1,"Hawksford, Mr. Walter James",2,4,0,0,16988,3,3,S,11,4,246,1
839,840,1,1,"Marechal, Mr. Pierre",2,4,0,0,11774,3,2,C,11,4,371,1
301,302,1,3,"McCoy, Mr. Bernard",2,4,2,0,367226,3,7,Q,11,4,380,2
712,713,1,1,"Taylor, Mr. Elmer Zebley",2,0,1,0,19996,1,2,S,11,4,596,2
726,727,1,2,"Renouf, Mrs. Peter Henry (Lillian Jefferys)",1,5,3,0,31027,3,7,S,12,2,502,2
489,490,1,3,"Coutts, Master. Eden Leslie ""Neville""",2,1,1,1,C.A. 37671,3,7,S,7,5,130,2
279,280,1,3,"Abbott, Mrs. Stanton (Rosa Hunt)",1,5,1,1,C.A. 2673,3,7,S,12,2,1,2


In [14]:
FP = differences[differences == 1].index
sample.iloc[FP,:]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,TitleKey,Surname,Family
278,279,0,3,"Rice, Master. Eric",0,1,4,1,382652,3,7,Q,7,5,505,5
14,15,0,3,"Vestrom, Miss. Hulda Amanda Adolfina",1,3,0,0,350406,0,7,S,8,3,626,1
474,475,0,3,"Strandberg, Miss. Ida Sofia",1,5,0,0,7553,2,7,S,8,3,586,1
642,643,0,3,"Skoog, Miss. Margit Elizabeth",0,1,3,2,347088,3,7,S,8,3,563,6
567,568,0,3,"Palsson, Mrs. Nils (Alma Cornelia Berglund)",1,5,0,4,349909,3,7,S,12,2,463,4
766,767,0,1,"Brewe, Dr. Arthur Jackson",2,4,0,0,112379,1,7,C,3,0,74,1
312,313,0,2,"Lahtinen, Mrs. William (Anna Sylfven)",1,5,1,1,250651,3,7,S,12,2,321,1
564,565,0,3,"Meanwell, Miss. (Marion Ogden)",1,4,0,0,SOTON/O.Q. 392087,2,7,S,8,3,389,1
501,502,0,3,"Canavan, Miss. Mary",1,5,0,0,364846,0,7,Q,8,3,92,1
113,114,0,3,"Jussila, Miss. Katriina",1,5,1,0,4136,2,7,S,8,3,296,3


In [15]:
# Do this for the test set
dc.transform_features(data)
dc.encode_features(data)
X = data[quick_features]
predictions =logreg.predict(X)

# Save into output file
ids = data['PassengerId']

output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': predictions })
output.to_csv('titanic-predictions_logistic.csv', index = False)
output.head()



DataCleanup.ipynb:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "    for n in range(len(Surname)):\n",


NameError: name 'logreg' is not defined